<a href="https://colab.research.google.com/github/ze-16/Speech-training-agent/blob/main/Speech_training_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import gdown
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertModel
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor
import timeit

In [ ]:
# Coding the file path and allowing access through link
file_id = '1mC-mj6zuojm8jWRBH46Vj2Gno1ZsXlRF'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Text_data.csv'

In [ ]:
# If statement to make sure file is only downloaded once and not repeatedly
if not os.path.exists(output):
    print("Downloading file...")
    gdown.download(url, output, quiet=False)
else:
    print(f"File already exists.")

File already exists.


In [ ]:
# Loading the goemotions_2 dataset for EDA
df = pd.read_csv(output)

In [ ]:
# Loading the dimensions of the dataset
df.shape

(70000, 37)

In [ ]:
# Loading the first 10 values of the dataset to explore what the data looks like
df.head(10)

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,We can hope,ee3o3ko,darkenseyreth,EdmontonOilers,t3_ag4r9j,t1_ee3mhad,1.547529e+09,62,False,0,...,0,0,1,0,0,0,0,0,0,0
1,Shhh don't give them the idea!,eebl3z7,BoinkBoinkEtAliae,MurderedByWords,t3_ah3o76,t1_eeb68lo,1.547777e+09,76,False,0,...,0,0,0,0,0,0,0,0,0,0
2,"Thank you so much, kind stranger. I really nee...",ed4fe9l,savageleaf,raisedbynarcissists,t3_abwh00,t1_ed4etbj,1.546482e+09,24,False,0,...,0,0,0,0,0,0,0,0,0,0
3,Ion know but it would be better for you to jus...,efavtdu,CADDiLLXC,darknet,t3_al4njw,t3_al4njw,1.548800e+09,62,False,0,...,0,0,0,0,0,0,0,0,0,1
4,I'm honestly surprised. We should have fallen ...,ee2imz2,CorporalThornberry,CollegeBasketball,t3_afxt6t,t1_ee22nyr,1.547497e+09,55,False,0,...,0,0,0,0,0,0,0,0,0,0
5,Jurisprudence fetishist gets off on technicality.,ednv5fy,100Gold,TopMindsOfReddit,t3_ae3p1m,t1_ednn3pz,1.547068e+09,12,False,0,...,0,0,0,0,0,0,0,0,0,1
6,One of my worst nightmares. Ugh.,ed60o6e,June1111,ExpectationVsReality,t3_ac7y4f,t3_ac7y4f,1.546541e+09,55,False,0,...,0,0,0,0,0,0,0,0,0,0
7,Is this real? 🤔,edjldcm,Uneekusername101,instant_regret,t3_adr232,t3_adr232,1.546928e+09,72,False,0,...,0,0,0,0,0,0,0,0,0,0
8,My favorite pod when they are fresh & clear. S...,ee7j34k,_BAMBOOZLE_,juul,t3_agn37x,t3_agn37x,1.547662e+09,12,False,0,...,0,0,0,0,0,0,0,0,0,1
9,I get wound up over completely imagined conver...,eeztbjr,TheLiquidSquid,aspergers,t3_ajpzf0,t1_eeypbzh,1.548483e+09,61,False,0,...,0,0,1,0,1,0,0,0,0,0


In [ ]:
# Check the names of the headings and explore their data-types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   text                  70000 non-null  object 
 1   id                    70000 non-null  object 
 2   author                70000 non-null  object 
 3   subreddit             70000 non-null  object 
 4   link_id               70000 non-null  object 
 5   parent_id             70000 non-null  object 
 6   created_utc           70000 non-null  float64
 7   rater_id              70000 non-null  int64  
 8   example_very_unclear  70000 non-null  bool   
 9   admiration            70000 non-null  int64  
 10  amusement             70000 non-null  int64  
 11  anger                 70000 non-null  int64  
 12  annoyance             70000 non-null  int64  
 13  approval              70000 non-null  int64  
 14  caring                70000 non-null  int64  
 15  confusion          

In [ ]:
# Basic descriptive statistics for categorical variables
df.describe(include='object')

,text,id,author,subreddit,link_id,parent_id
count,70000,70000,70000,70000,70000,70000
unique,43293,43448,37837,483,34995,41480
top,Thank you.,eep131n,[deleted],cringe,t3_ae6ejj,t3_ae6ejj
freq,18,5,1236,295,35,25


In [ ]:
# Checking for any missing values and their sum for each column
df.isna().sum()

,0
text,0
id,0
author,0
subreddit,0
link_id,0
parent_id,0
created_utc,0
rater_id,0
example_very_unclear,0
admiration,0


In [ ]:
# Removing variables/columns that will be of no consequence to the result upon initial look
df = df.drop(columns=(['id','author','subreddit','link_id','parent_id','created_utc','rater_id','example_very_unclear']))

In [ ]:
# Running a check to make sure the targetted columns have been removed
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            70000 non-null  object
 1   admiration      70000 non-null  int64 
 2   amusement       70000 non-null  int64 
 3   anger           70000 non-null  int64 
 4   annoyance       70000 non-null  int64 
 5   approval        70000 non-null  int64 
 6   caring          70000 non-null  int64 
 7   confusion       70000 non-null  int64 
 8   curiosity       70000 non-null  int64 
 9   desire          70000 non-null  int64 
 10  disappointment  70000 non-null  int64 
 11  disapproval     70000 non-null  int64 
 12  disgust         70000 non-null  int64 
 13  embarrassment   70000 non-null  int64 
 14  excitement      70000 non-null  int64 
 15  fear            70000 non-null  int64 
 16  gratitude       70000 non-null  int64 
 17  grief           70000 non-null  int64 
 18  joy   

In [ ]:
# Upon further reflection, dictionary created to condense the range of emotions, for the sake of simplicity of providing feedback at project end
emotion_groups = {
    'anger': ['anger', 'annoyance', 'disapproval', 'disgust', 'fear', 'nervousness'],
    'joy': ['joy', 'admiration', 'amusement', 'approval', 'caring', 'desire',
            'excitement', 'gratitude', 'love', 'optimism', 'pride', 'relief'],
    'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
    'neutral': ['neutral', 'confusion', 'curiosity', 'realization', 'surprise']
}

In [ ]:
# Function that helps map the 28 emotions into 4 core groups, in order of priority
def get_core_emotion(row):
  for col in emotion_groups['anger']:
        if row[col] == 1:
            return 'anger'

  for col in emotion_groups['sadness']:
        if row[col] == 1:
            return 'sadness'

  for col in emotion_groups['joy']:
        if row[col] == 1:
            return 'joy'

  return 'neutral'

In [ ]:
# Applying the above function onto the dataset
df['core_emotion'] = df.apply(get_core_emotion, axis=1)

In [ ]:
# Checking to see the distribution of each category
df['core_emotion'].value_counts()

,count
core_emotion,
neutral,26126
joy,25520
anger,12900
sadness,5454


In [ ]:
# Writing the new organised emotions into a new csv file
df[['text', 'core_emotion']].to_csv('Text_data2.csv', index=False)

In [ ]:
# Loading the new file and checking if the mapping worked
df = pd.read_csv('Text_data2.csv')
df.head()

,text,core_emotion
0,We can hope,joy
1,Shhh don't give them the idea!,anger
2,"Thank you so much, kind stranger. I really nee...",joy
3,Ion know but it would be better for you to jus...,neutral
4,I'm honestly surprised. We should have fallen ...,joy


In [ ]:
# Checking to see the distribution of each category
df['core_emotion'].value_counts()

,count
core_emotion,
neutral,26126
joy,25520
anger,12900
sadness,5454


In [ ]:
# Mapping the data to integers for the model.
label_map = {'neutral': 0, 'joy': 1, 'sadness': 2, 'anger': 3}
df['core_emotion'] = df['core_emotion'].map(label_map)

In [ ]:
# Loading a pre-trained bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Function to tokenize raw text data for BERT
def TokenizetoBERT(dataframe, tokenizer, max_len=128):
  # Loading the data and labelling in regards to tensors
  text = dataframe['text'].tolist()
  labels = dataframe['core_emotion']
  labels_tensor = torch.tensor(dataframe['core_emotion'].values)

  # Tokenize
  tokenizing = tokenizer(
      text,
      add_special_tokens=True,
      max_length=max_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
  )

  # Getting the tensors
  input_ids = tokenizing['input_ids']
  attention_masks = tokenizing['attention_mask']

  # Returning the tensors
  return input_ids, attention_masks, labels_tensor

In [ ]:
# Splitting the data based on variable 'core_emotion'
train_set,test_set = train_test_split(df,test_size=0.3, random_state=14,stratify=df['core_emotion'])

In [ ]:
# Using the previously defined function to tokenize
train_input, train_mask, train_label = TokenizetoBERT(train_set, tokenizer)
test_input, test_mask, test_label = TokenizetoBERT(test_set, tokenizer)

# Turning tensors into dataset
train_dataset = TensorDataset(train_input, train_mask, train_label)
test_dataset = TensorDataset(test_input, test_mask, test_label)

In [ ]:
# Using the built in DataLoader class from pytorch
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=2,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=64,num_workers=2,pin_memory=True)

In [ ]:
# Initialising BERT in order to check for configurations
bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Checking for configurations for BERT
config = bert.config
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.6",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
# Deleting BERT and config and cleaning up memory
del bert
del config
torch.cuda.empty_cache()

In [ ]:
# Configuring BERT model to my choice using a python class
class BertClassifier(nn.Module):
  def __init__(self, n_classes):
    super(BertClassifier, self).__init__()

    self.bert = BertModel.from_pretrained('bert-base-uncased')

    self.drop = nn.Dropout(p=0.3)

    self.dim_reduction = nn.Linear (768,32)

    self.activation = nn.ReLU()

    # Defining the final classifier
    self.classifier = nn.Linear (32,n_classes)

  def forward(self, input_ids, attention_mask):
        # Feeding tokenized text into BERT
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        pooled_outputs = outputs.pooler_output
        output = self.drop(pooled_outputs)

        reduced = self.dim_reduction(output)

        reduced = self.activation(reduced)

        # Making the final prediction
        return self.classifier(reduced)

In [ ]:
# Checking whether colab is utilising CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Creating instance of previous defined class/initialising the model and feeding n_classes=4 as a parameter
model = BertClassifier(n_classes=4)

# Transfers the model to "device"/GPU
model = model.to(device)

# Compiling the model for increasing training speed
model = torch.compile(model)

Using device: cuda


In [ ]:
# Setting the optimizer with learning rate 3x10^-5
optimizer = AdamW(model.parameters(), lr=3e-5)


# Setting the weights of each class depending on the ratios between classes
weights = torch.tensor([1.0, 1.0, 2.0, 1.5]).to(device)

# Pre defined function calculating the loss using the weights defined above
loss_fn = nn.CrossEntropyLoss(weight=weights)

In [ ]:
# Defining the number of epochs (training iterations) and variable to store the accuracy for each epoch
epochs = 3
best_accuracy = 0

In [ ]:
# Scheduler to help slow down the learning rate during training
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_loader)*epochs
)

In [ ]:
# Function defined for the training loop
def train_loop(train_loader, model, optimizer, loss_fn,scheduler):
    size = len(train_loader.dataset)

    # Switching the model to training mode
    model.train()


    # Loop to make predictions and adjust weights
    for i, batch in enumerate(train_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Making predictions
        pred = model(input_ids, attention_mask)
        loss = loss_fn(pred, labels)

        # Back propogating and adjusting gradients
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # Adding scheduler step so the learning rate slows during trainign to prevent overfitting
        scheduler.step()

        if i % 300 == 0:
          loss, current = loss.item(), i * train_loader.batch_size + len(input_ids)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# Function defined for the testing loop
def test_loop(test_loader, model, loss_fn):

    # Switching model to evaluation mode
    model.eval()
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    # Evaluating the model, no gradients are calculated
    with torch.no_grad():
        for batch in test_loader:

            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            # Making predictions
            pred = model(input_ids,attention_mask)
            test_loss += loss_fn(pred, labels).item()
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# Single most useful line of code in terms of training speed, using A100 gpu and batch size of 64, it brought training time down from ~14m to ~4m
torch.set_float32_matmul_precision('high')

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [ ]:
# Loop to train and test through epochs
start = timeit.default_timer()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, optimizer, loss_fn,scheduler)
    test_loop(test_loader, model, loss_fn)
print("Done!")
end = timeit.default_timer()
print(f"Total time taken: {end - start}")

Epoch 1
-------------------------------
loss: 1.454850  [   64/49000]
loss: 0.932265  [19264/49000]
loss: 1.086041  [38464/49000]


W0202 14:57:28.330000 8361 torch/fx/experimental/symbolic_shapes.py:6833] [0/1] _maybe_guard_rel() was called on non-relation expression Eq(s43, 1) | Eq(s72, s43)
W0202 14:57:36.235000 8361 torch/fx/experimental/symbolic_shapes.py:6833] [0/2] _maybe_guard_rel() was called on non-relation expression Eq(s43, 1) | Eq(s72, s43)


Test Error: 
 Accuracy: 62.9%, Avg loss: 0.941681 

Epoch 2
-------------------------------
loss: 0.581207  [   64/49000]
loss: 0.848612  [19264/49000]
loss: 1.047168  [38464/49000]
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.938189 

Epoch 3
-------------------------------
loss: 0.825429  [   64/49000]
loss: 0.721616  [19264/49000]
loss: 0.540363  [38464/49000]
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.978354 

Done!
Total time taken: 197.8699818120001


In [ ]:
# Initialising the feature extractor for wav2vec2
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-er")

In [ ]:
# Building a class for audio processing, using a variant of wav2vec2 speciic for 4 emotion classification
class Wav2Model (nn.Module):
  def __init__(self):
    super(Wav2Model, self).__init__()

    # Initialising the model with the pretrained weights
    self.wav2vec2 = Wav2Vec2Model.from_pretrained("superb/wav2vec2-base-superb-er")

    # Freezing the model so it doesnt go into a training run
    for param in self.wav2vec2.parameters():
      param.requires_grad = False


    self.dim_reduction = nn.Linear(768, 32)

    self.activation = nn.ReLU()

  def forward(self, input_values):
    # With torch.no_grad because I will not be adjusting the gradients
    with torch.no_grad():
      outputs = self.wav2vec2(input_values)
      pooled_outputs = outputs.last_hidden_state.mean(dim=1)

      output = self.dim_reduction(pooled_outputs)
      output = self.activation(output)
      return output

